<a href="https://colab.research.google.com/github/diegoOrd123/Examen-Primer-parcial/blob/main/Primer_parcial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Importamos las librerias necesarias

In [3]:
import os
import numpy as np
import pandas as pd
from matplotlib import pyplot

In [5]:
input_layer_size  = 14
num_labels = 5
data = pd.read_csv('/content/drive/MyDrive/Dataset/Base.csv', delimiter=';')
print(data.shape)

data.info()

(1000000, 14)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 14 columns):
 #   Column                        Non-Null Count    Dtype  
---  ------                        --------------    -----  
 0   income                        1000000 non-null  float64
 1   name_email_similarity         1000000 non-null  float64
 2   prev_address_months_count     1000000 non-null  int64  
 3   current_address_months_count  1000000 non-null  int64  
 4   customer_age                  1000000 non-null  int64  
 5   intended_balcon_amount        1000000 non-null  float64
 6   zip_count_4w                  1000000 non-null  int64  
 7   velocity_6h                   1000000 non-null  float64
 8   velocity_24h                  1000000 non-null  float64
 9   velocity_4w                   1000000 non-null  float64
 10  credit_risk_score             1000000 non-null  int64  
 11  bank_months_count             1000000 non-null  int64  
 12  proposed_credit

In [7]:
y = data.iloc[:, -1]
print(y)
mapping = {'AB': 0, 'AA': 1, 'AC': 2, 'AE': 3, 'AD': 4}
def convert_column_to_numeric(data, mapping):
    vectorized_map = np.vectorize(mapping.get)
    numeric_data = vectorized_map(data)
    return numeric_data
numeric_data = convert_column_to_numeric(y, mapping)
y = numeric_data
print(y)

0         AA
1         AD
2         AB
3         AB
4         AA
          ..
999995    AB
999996    AC
999997    AA
999998    AB
999999    AA
Name: payment_type, Length: 1000000, dtype: object
[1 4 0 ... 1 0 1]


In [11]:
X=data.iloc[:,:-1]
print(X.shape)
def income_group(income):
    if income < 3:
        return 'Low'
    elif 3 <= income < 7:
        return 'Medium'
    else:
        return 'High'
X['income_level'] = X['income'].apply(income_group)
print(X.shape)
print(X)

(1000000, 13)
(1000000, 14)
              income  name_email_similarity  prev_address_months_count  \
0       3.000000e+00           9.865060e+14                         -1   
1       8.000000e+00           6.174260e+15                         -1   
2       8.000000e+00           9.967070e+15                          9   
3       6.000000e+15           4.751000e+15                         11   
4       9.000000e+00           8.423070e+15                         -1   
...              ...                    ...                        ...   
999995  8.000000e+00           1.246900e+14                         -1   
999996  9.000000e+00           8.245440e+15                         -1   
999997  8.000000e+00           1.408910e+15                         -1   
999998  9.000000e+00           2.480240e+13                         52   
999999  6.000000e+15           9.933910e+15                         -1   

        current_address_months_count  customer_age  intended_balcon_amount  \
0    